In [8]:
!pip install spacy
!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 5.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.0/636.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 894.7/894.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 6.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 6.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 5.4 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# cleaning the data and saving as json in chunks

In [1]:
import fitz  # PyMuPDF

def extract_clean_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text


In [2]:
import re

def clean_text(text):
    # Remove headers/footers if repetitive (e.g., page numbers, common titles)
    text = re.sub(r'\nPage \d+\n', ' ', text)  # Example: Page numbers
    text = re.sub(r'\s+', ' ', text)           # Replace multiple spaces/newlines
    return text.strip()


In [3]:
text1 = extract_clean_text('initial_data/AI Training Document.pdf')
text2 = clean_text(text1)

In [7]:
print(text2[:500])  # Print the first 500 characters of the cleaned text

User Agreement 1. Introduction This User Agreement, the Mobile Application Terms of Use, and all policies and additional terms posted on and in our sites, applications, tools, and services (collectively "Services") set out the terms on which eBay offers you access to and use of our Services. You can find an overview of our policies here. The Mobile Application Terms of Use, all policies, and additional terms posted on and in our Services are incorporated into this User Agreement. You agree to co


## sentence aware chunking

In [ ]:
# import nltk
# from nltk.tokenize import sent_tokenize
# nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ishantkamboj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:

# def chunk_text(text, min_words=100, max_words=300):
#     sentences = sent_tokenize(text)
#     chunks = []
#     current_chunk = ""
#     current_words = 0

#     for sentence in sentences:
#         word_count = len(sentence.split())
#         if current_words + word_count <= max_words:
#             current_chunk += " " + sentence
#             current_words += word_count
#         else:
#             if current_words >= min_words:
#                 chunks.append(current_chunk.strip())
#                 current_chunk = sentence
#                 current_words = word_count
#             else:
#                 current_chunk += " " + sentence
#                 current_words += word_count

#     if current_chunk:
#         chunks.append(current_chunk.strip())

#     return chunks


In [9]:
import spacy

# Load the small English model
nlp = spacy.load("en_core_web_sm")

def chunk_text_spacy(text, min_words=100, max_words=300):
    doc = nlp(text)
    chunks = []
    current_chunk = ""
    current_words = 0

    for sent in doc.sents:
        sent_text = sent.text.strip()
        word_count = len(sent_text.split())

        if current_words + word_count <= max_words:
            current_chunk += " " + sent_text
            current_words += word_count
        else:
            if current_words >= min_words:
                chunks.append(current_chunk.strip())
                current_chunk = sent_text
                current_words = word_count
            else:
                current_chunk += " " + sent_text
                current_words += word_count

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks


In [10]:
# saving for vector db

chunks = chunk_text_spacy(clean_text(extract_clean_text("initial_data/AI Training Document.pdf")))

# Save to JSON/CSV or directly embed
import json
with open("chunks.json", "w") as f:
    json.dump(chunks, f, indent=2)
